In [1]:
from Nb_utils import *

NOTE! Installing ujson may make loading annotations faster.


/home/superorange5/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


## set test data

### a. bdd

In [10]:
dataset_name = 'VOC2007_bddvalsmall'
#dataset_name = 'VOC2007_cityval1'
cfg = setup("configs/pt/smallc2b.yaml")


### b. cityscape

In [2]:
cfg = setup("configs/pt/final_k2c.yaml")
dataset_name = 'VOC2007_cityval1'

In [3]:

data_annotation = DatasetCatalog.get(dataset_name)

box_list=[]
for ann in data_annotation[0]['annotations']:
    box_list.append(ann['bbox'])
bboxes_gt = structures.Boxes(torch.Tensor(box_list)).to("cuda")


In [4]:

test_data_loader = build_detection_test_loader(cfg, dataset_name)
data_loader_iter = iter(test_data_loader)
test_data = data_loader_iter.next()
target_metadata = MetadataCatalog.get(dataset_name)

## pick GT

In [5]:
len(bboxes_gt)

18

In [10]:
bboxes_to_draw = bboxes_gt[[9,13,14,15],:]

In [11]:
drawbb(test_data[0]['file_name'], target_metadata, bboxes_to_draw)

## prediction

### a. ck2b

In [29]:
#--------ck2bdd-------
model_c2b = get_model('c2b', 11999)
model_k2b = get_model('k2b', 'final')
model_list_ck2b=[model_c2b,model_k2b]

keep_experiments/c2b/cfg.yaml
keep_experiments/c2b/model_0011999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2b/cfg.yaml
keep_experiments/k2b/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [33]:

proposals_roih_c2b = get_proposal_roih(test_data,model_c2b)
proposals_roih_k2b = get_proposal_roih(test_data,model_k2b)

c2b_prediction = scaling(proposals_roih_c2b)

new_boxes = c2b_prediction[[9,20,24,25,26,29,30,36],:]



### b.skf2c

In [5]:
cfg = setup("configs/pt/final_k2c.yaml")
dataset_name = 'VOC2007_cityval1'
test_data_loader = build_detection_test_loader(cfg, dataset_name)


#---------skf2c-------
model_s2c = get_model('s2c', 15999)
model_k2c = get_model('k2c', 19999)
model_f2c = get_model('f2c', 'final') #8class
model_list_skf2c=[model_k2c,model_f2c]

data_loader_iter = iter(test_data_loader)
test_data = data_loader_iter.next()


source_list =['sim10k','kitti','foggycityscape']

keep_experiments/s2c/cfg.yaml
keep_experiments/s2c/model_0015999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2c/cfg.yaml
keep_experiments/k2c/model_0019999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/f2c/cfg.yaml
keep_experiments/f2c/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [80]:

proposals_roih_s2c = get_proposal_roih(test_data,model_s2c)



In [81]:
s2c_prediction = scaling(proposals_roih_s2c,1.7066)


In [83]:
s2c_prediction[0:13]

Boxes(tensor([[ 146.5384,  385.5361,  448.0758,  536.4097],
        [1564.1022,  259.5203, 2047.9199,  777.3470],
        [1327.8187,  367.4783, 1659.7887,  568.8662],
        [1123.5488,  396.7930, 1204.7117,  453.3259],
        [ 836.7457,  392.7574,  934.2112,  448.4383],
        [   0.0000,  362.3014,   79.4530,  587.5843],
        [ 696.5767,  368.8785,  784.0192,  442.3321],
        [1329.2263,  380.3962, 1444.2534,  543.8480],
        [1087.3143,  396.1635, 1141.2175,  445.6487],
        [ 763.1011,  375.6505,  825.3354,  435.2645],
        [ 120.4187,  343.5235,  211.7196,  433.3668],
        [1048.0598,  367.9155, 1106.7322,  426.8443],
        [1586.8326,  369.5982, 1912.3317,  733.5095]], device='cuda:0'))

In [132]:
def drawbb_byindex(image_filename, target_metadata, bboxes_to_draw, box_index,score):
    im = cv2.imread(image_filename, cv2.IMREAD_COLOR)[:, :, ::-1]
    v = Visualizer(
            im[:, :, ::-1], 
            metadata=target_metadata, 
            scale=1,
            )
    for index_slice in box_index:
        

        for idx, box in enumerate(bboxes_to_draw[index_slice]):
            box = box.to('cpu')
            #print(index_slice[idx])
            idx_exact = index_slice[idx]
        #for idx_np in index_slice:
        #    idx = int(idx_np)
        #    box = bboxes_to_draw[idx].to('cpu')
            #print(box)
            v.draw_box(box,edge_color='r')
            v.draw_text("{}: score={:.2f}".format(idx_exact,score[idx_exact]), tuple(box[:2].numpy()),color='r')

    v = v.get_output()
    img =  v.get_image()[:, :, ::-1]
    cv2.imwrite('showbb.jpg', img)

In [133]:
box_index = [np.arange(11,13),np.arange(15,25),[27,29,30,31,33,42]]


In [136]:
box_index = [[14,26,28,32]]#np.arange(34,41),np.arange(43,49),]


In [137]:
scores = proposals_roih_s2c[0].get('scores')
drawbb_byindex(test_data[0]['file_name'], target_metadata, s2c_prediction, box_index,scores)

In [29]:

proposals_roih_s2c[0].get('scores')

tensor([0.9896, 0.9788, 0.9741, 0.9658, 0.9640, 0.9374, 0.9211, 0.9134, 0.9056,
        0.8670, 0.8262, 0.7647, 0.5630, 0.1342, 0.1194, 0.1167, 0.0587, 0.0585,
        0.0536, 0.0536, 0.0522, 0.0448, 0.0442, 0.0348, 0.0340, 0.0339, 0.0326,
        0.0323, 0.0317, 0.0304, 0.0275, 0.0264, 0.0256, 0.0249, 0.0196, 0.0190,
        0.0184, 0.0180, 0.0177, 0.0174, 0.0167, 0.0156, 0.0147, 0.0129, 0.0123,
        0.0106, 0.0096, 0.0094, 0.0084, 0.0052], device='cuda:0')